In [8]:
# Imports
import os
import numpy as np
import pandas as pd
from PIL import Image
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from keras.models import load_model
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras.utils import Sequence
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from city_sustainability.models import unet_model
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping
from city_sustainability.quality import life_quality
from tensorflow.keras.applications.vgg16 import VGG16
from city_sustainability.preprocessing import image_resize
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from city_sustainability.loading_data_448x448 import loading_paths_448
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from city_sustainability.loading_data_448x448 import image_and_label_arrays_448
from tensorflow.keras.layers import Conv2D, BatchNormalization, Dropout, MaxPooling2D, UpSampling2D, Concatenate, Input

In [2]:
data_path ='../../raw_data/OpenEarthMap_wo_xBD'
image_paths, label_paths = loading_paths(data_path)

image_datagen = ImageDataGenerator(validation_split=0.2)
label_datagen = ImageDataGenerator(validation_split=0.2)

seed = 28
batch_size = 64

image_df = pd.DataFrame(image_paths, columns=["filename"])
label_df = pd.DataFrame(label_paths, columns=["filename"])

image_generator = image_datagen.flow_from_dataframe(
    dataframe=image_df,
    directory=None,
    x_col="filename",
    class_mode=None,
    batch_size=batch_size,
    seed=seed,
    color_mode='rgb',
    target_size=(448, 448),
    subset = 'training',
    shuffle=False)

label_generator = label_datagen.flow_from_dataframe(
    dataframe=label_df,
    directory=None,
    x_col="filename",
    class_mode=None,
    batch_size=batch_size,
    seed=seed,
    color_mode='grayscale',
    target_size=(448, 448),
    subset = 'training',
    shuffle=False)

validation_image_generator = image_datagen.flow_from_dataframe(
    dataframe=image_df,
    directory=None,
    x_col="filename",
    class_mode=None,
    batch_size=batch_size,
    seed=seed,
    color_mode='rgb',
    target_size=(448, 448),
    subset = 'validation',
    shuffle=False)

validation_label_generator = label_datagen.flow_from_dataframe(
    dataframe=label_df,
    directory=None,
    x_col="filename",
    class_mode=None,
    batch_size=batch_size,
    seed=seed,
    color_mode='grayscale',
    target_size=(448, 448),
    subset = 'validation',
    shuffle=False)

train_generator = zip(image_generator, label_generator)
validation_generator = zip(validation_image_generator, validation_label_generator)

def preprocess_labels(labels):
    categorical_labels = to_categorical(labels, num_classes=9)
    return categorical_labels

train_generator = ((images, preprocess_labels(labels)) for images, labels in train_generator)
validation_generator = ((images, preprocess_labels(labels)) for images, labels in validation_generator)

Found 2150 validated image filenames.
Found 2150 validated image filenames.
Found 537 validated image filenames.
Found 537 validated image filenames.


In [3]:
# Create the model
model_full = unet_model.build_vgg16_model_full(input_shape=(448, 448, 3), num_classes = 9)

2023-05-29 12:42:26.938059: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-05-29 12:42:26.940647: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [4]:
# Total params: 26,482,249
# Trainable params: 11,764,617
# Non-trainable params: 14,717,632
model_full.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 448, 448, 3  0           []                               
                                )]                                                                
                                                                                                  
 block1_conv1 (Conv2D)          (None, 448, 448, 64  1792        ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 block1_conv2 (Conv2D)          (None, 448, 448, 64  36928       ['block1_conv1[0][0]']           
                                )                                                             

                                                                                                  
 batch_normalization_2 (BatchNo  (None, 112, 112, 25  1024       ['conv2d_2[0][0]']               
 rmalization)                   6)                                                                
                                                                                                  
 dropout_2 (Dropout)            (None, 112, 112, 25  0           ['batch_normalization_2[0][0]']  
                                6)                                                                
                                                                                                  
 up_sampling2d_3 (UpSampling2D)  (None, 224, 224, 25  0          ['dropout_2[0][0]']              
                                6)                                                                
                                                                                                  
 concatena

In [5]:
# Compile the model
unet_model.compile_model(model_full)

In [9]:
path = '../../model_training/marcel/20230529-06-unet_vgg16full_1.00img_16epch_64btch_0.57acc'
model_full = load_model(path)

In [10]:
# Train the model on the current batch
model_full.fit(train_generator, 
               epochs=50,
               steps_per_epoch= image_generator.samples//batch_size,
               validation_data= validation_generator,
               validation_steps= validation_image_generator.samples//batch_size)

Epoch 1/50


2023-05-29 12:50:10.196874: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
2023-05-29 12:50:34.823381: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 6576668672 exceeds 10% of free system memory.
2023-05-29 12:50:35.221948: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 9865003008 exceeds 10% of free system memory.
2023-05-29 12:50:44.204845: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 9865003008 exceeds 10% of free system memory.
2023-05-29 12:51:05.337809: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 6576668672 exceeds 10% of free system memory.


 1/33 [..............................] - ETA: 48:07 - loss: 1.1599 - accuracy: 0.5583

2023-05-29 12:52:05.817454: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 6576668672 exceeds 10% of free system memory.


33/33 [==============================] - ETA: 0s - loss: 1.1498 - accuracy: 0.5705  

2023-05-29 13:38:09.691010: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


33/33 [==============================] - 3113s 94s/step - loss: 1.1498 - accuracy: 0.5705 - val_loss: 1.2300 - val_accuracy: 0.5397
Epoch 2/50
11/33 [=========>....................] - ETA: 32:10 - loss: 1.1596 - accuracy: 0.5669

KeyboardInterrupt: 

In [12]:
model_full.save('../../model_training/marcel/20230529-06-unet_vgg16full_1.00img_50epch_64btch')

2023-05-29 09:19:25.539485: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,?,?,?]
	 [[{{node inputs}}]]
2023-05-29 09:19:25.574510: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,?,?,?]
	 [[{{node inputs}}]]
2023-05-29 09:19:25.600368: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,?,?,?]
	 [[{{node inputs}}]]
2023-05-29

INFO:tensorflow:Assets written to: ../../model_training/marcel/20230529-06-unet_vgg16full_1.00img_50epch_64btch/assets


INFO:tensorflow:Assets written to: ../../model_training/marcel/20230529-06-unet_vgg16full_1.00img_50epch_64btch/assets
